In [3]:
import torch
from torch import nn

## 理解神经网络

- 从shape的含义，shape 的变化，可能是理解神经网络模型结构乃至处理过程的一个很实用的切入。

## `nn.utils.clip_grad_norm_`

- 避免梯度爆炸
   
    - 计算所有梯度的范数：这一步骤涉及到计算网络中所有参数的梯度范数。通常使用 L2 范数，即平方和的平方根。
        - 也是 `nn.utils.clip_grad_norm_` 的返回值；
    - 比较梯度范数与最大值：将计算出的梯度范数与设定的最大范数值比较。
    - 裁剪梯度：如果梯度范数大于最大值，那么将所有梯度缩放到最大值以内。这是通过乘以一个缩放因子实现的，缩放因子是最大范数值除以梯度范数。
    - 更新梯度：使用裁剪后的梯度值更新网络参数。

- 一般training过程中用到：

    ```
    optimizer.zero_grad()
    # 反向传播计算 parameters 的 grad
    loss.backward()
    # 计算完梯度之后，norm 所有参数的 grads
    nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
    # 基于更新后的 grad 值来更新参数
    optimizer.step()
    ```

In [13]:
# 定义一个简单的网络及其梯度
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.param1 = torch.nn.Parameter(torch.tensor([3.0, 4.0]))
        self.param2 = torch.nn.Parameter(torch.tensor([1.0, 2.0]))

# 创建网络实例
net = SimpleNet()

# 设置梯度
net.param1.grad = torch.tensor([3.0, 4.0])
net.param2.grad = torch.tensor([1.0, 2.0])

# 最大梯度范数
max_norm = 5.0

In [14]:
# 计算梯度范数
total_norm = torch.sqrt(sum(p.grad.norm()**2 for p in net.parameters()))

# 计算缩放因子
scale = max_norm / (total_norm + 1e-6)

# 应用梯度裁剪
for p in net.parameters():
    p.grad.data.mul_(scale)

# 更新后的梯度
clipped_grads = [p.grad for p in net.parameters()]

total_norm, clipped_grads

(tensor(5.4772), [tensor([2.7386, 3.6515]), tensor([0.9129, 1.8257])])

In [15]:
# 定义一个简单的网络及其梯度
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.param1 = torch.nn.Parameter(torch.tensor([3.0, 4.0]))
        self.param2 = torch.nn.Parameter(torch.tensor([1.0, 2.0]))

# 创建网络实例
net = SimpleNet()

# 设置梯度
net.param1.grad = torch.tensor([3.0, 4.0])
net.param2.grad = torch.tensor([1.0, 2.0])

# 最大梯度范数
max_norm = 5.0

In [16]:
nn.utils.clip_grad_norm_(net.parameters(), max_norm)

tensor(5.4772)

In [17]:
for p in net.parameters():
    print(p.grad)

tensor([2.7386, 3.6515])
tensor([0.9129, 1.8257])
